In [136]:
#~ Imports & globals
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
plot_jsons = {}
def toInt(x):
    try: return int(x)
    except: return x

In [311]:
#~ Read & organize data
C = pd.read_csv("../data/full_data.csv")
C = C.loc[C.location!='World']
C['date'] = pd.to_datetime(C['date'])
C = C.set_index(['date','location']).unstack(fill_value=0). \
      asfreq('D',fill_value=0).stack().sort_index(level=1).reset_index()

C_ = C.groupby('location')[['new_cases','new_deaths']].rolling(5).mean().applymap(toInt).reset_index(0,drop=True)
C  = C.join(C_,rsuffix='_avg').dropna()
C['DATE']= C.date.dt.strftime("%m-%d")
C['NiceDate']= C.date.dt.strftime("%B %d")

In [312]:
#~ Figure 1
traces = []
for country, cdf in C.groupby('location'):
    if cdf['total_cases'].max()>50000:
        trace = go.Scatter(x=cdf['date'],y=cdf['new_cases_avg'],name=country, 
                          hovertemplate= country+"<br>%{x}: <b>%{y}</b><br><extra></extra>")
        traces.append(trace)

layout = go.Layout(title_text="New Cases Over Time",
                  xaxis_title_text="Date",
                  yaxis_title_text="New Cases (among test results reported)",
                  yaxis_type="log")
fig = go.Figure(data=traces, layout=layout)
fig.show()

In [313]:
fig.write_json("../data/json/new_cases_by_country.json")

In [378]:
#~ Figure 2
traces = []
for country, cdf in C.loc[C.total_cases>100].groupby('location'):
    if cdf['total_cases'].max()>20000 or country=="South Korea":
        trace = go.Scatter(x=cdf['total_cases'],y=cdf['new_cases_avg'],name=country,
                          customdata=cdf['NiceDate'], mode='lines',
                          hovertemplate= country+"<br>Date: %{customdata}<br>Total: %{x}<br><b>New: %{y}</b><extra></extra>")
        traces.append(trace)

layout = go.Layout(title_text="New Cases vs Total Cases",
                  xaxis_title_text="Total Cases",
                  yaxis_title_text="New Cases (among test results reported)",
                  yaxis_type="log", xaxis_type="log")
fig = go.Figure(data=traces, layout=layout)
fig.show()

In [81]:
fig.write_json("../data/json/new_vs_total.json")

In [100]:
# pd.date_range(C.date.min(),C.date.max(),freq='D')
# Cusa = C.loc[C.location=='United States']
from datetime import datetime as dt

8585     January 03
8586     January 04
8587     January 05
8588     January 06
8589     January 07
8590     January 08
8591     January 09
8592     January 10
8593     January 11
8594     January 12
8595     January 13
8596     January 14
8597     January 15
8598     January 16
8599     January 17
8600     January 18
8601     January 19
8602     January 20
8603     January 21
8604     January 22
8605     January 23
8606     January 24
8607     January 25
8608     January 26
8609     January 27
8610     January 28
8611     January 29
8612     January 30
8613     January 31
8614    February 01
           ...     
8649       March 07
8650       March 08
8651       March 09
8652       March 10
8653       March 11
8654       March 12
8655       March 13
8656       March 14
8657       March 15
8658       March 16
8659       March 17
8660       March 18
8661       March 19
8662       March 20
8663       March 21
8664       March 22
8665       March 23
8666       March 24
8667       March 25


In [204]:
x=range(5)
list(range(1,len(x)-1))

[1, 2, 3]

In [206]:
C_['total_cases'][:4]

533     0
3112    7
4301    3
2861    6
Name: total_cases, dtype: int64

In [315]:
#~ Keep only the intersting countries & dates (Include South Korea & start in February)

C['final_cases']=C['location'].map(C.groupby('location')['total_cases'].max().to_dict())
C_ = C.loc[(C.final_cases>10200) & C['DATE'].apply(lambda x: int(x[:2])>1)].sort_values(by=['DATE'])
#df = px.data.gapminder()
fig = px.scatter(C_, x="total_cases", y="new_cases_avg", 
  animation_frame="DATE", animation_group="location", hover_name="location", 
                 text='location', range_y=[1,5], range_x=[2.5,5.6])
fig.update_traces(mode='markers', textposition='top center')
fig.update_layout(title_text="New Cases vs Total Cases",
                  xaxis_title_text="Total Cases",
                  yaxis_title_text="New Cases (among test results reported)",
                  yaxis_type="log", xaxis_type="log")
fig.show()

In [383]:
countries = list(set(C_['location']))
days = list(set(C_['DATE']))
traces = []
# dots = []
# t = list(range(len(countries)))
from itertools import chain
# tt = list(chain.from_iterable(zip(t,t)))
frames = [ {'data': [], 'traces': list(range(2*len(countries)))} for k in range(1, len(days)-1)]
# dotframes = [ {'data': [], 'traces': tt} for k in range(1, len(days)-1)]
for country, cdf in C_.groupby('location'):
    dct= dict(x=cdf['total_cases'],y=cdf['new_cases_avg'],name=country,
                      customdata=cdf['NiceDate'], line=dict(color='grey', width=1), mode='lines',
                      hovertemplate= country+"<br>Date: %{customdata}<br>Total: %{x}<br><b>New: %{y}</b><extra></extra>")
    traces.append(go.Scatter(dct))
#     traces.append(trace)
    dct['showlegend']=False
    traces.append(go.Scatter(dct))
#     dot = go.Scatter(x=cdf['total_cases'],y=cdf['new_cases_avg'],name=country,
#                       marker=dict(color='green', size=8), mode='marker')
#     dots.append(dot)
    for k, frame in enumerate(frames):
        datum = {'type':'scatter',
                 'x': cdf['total_cases'][:k+2],
                 'y': cdf['new_cases_avg'][:k+2]}
        dotum = {'type':'scatter',
                 'x': [cdf['total_cases'].iloc[k+1]],
                 'y': [cdf['new_cases_avg'].iloc[k+1]],
                 'mode':"markers+text", 'text':country, 'textposition':'top left',
                 'marker':dict(color="green", size=8)}
        frame['data'].append(datum)
        #frame['traces'].append(k+1)
        frame['data'].append(dotum)
        #frame['traces'].append(k+1)

layout = go.Layout(title_text="New Cases vs Total Cases",
                  xaxis_title_text="Total Cases",
                  yaxis_title_text="New Cases (among test results reported)",
                  yaxis_type="log", xaxis_type="log",
                  plot_bgcolor='white',
                  updatemenus=[dict(type='buttons', showactive=False,
                               y=1.05, x=1.15, xanchor='right', yanchor='top',
                               pad=dict(t=0, r=10),
                               buttons=[dict(label='Start',
                                             method='animate',
                                             args=[None, 
                                                   dict(frame=dict(duration=15, redraw=True),
                                                        transition=dict(duration=.5), fromcurrent=True,
                                                        mode='immediate')])])])

#~ Roughly log_10 of min & max of x and y
layout.update(xaxis =dict(range=[2.5,5.5], autorange=False, gridwidth=0.1, gridcolor='LightGrey'),
              yaxis =dict(range=[1,5], autorange=False, gridwidth=0.1, gridcolor='LightGrey'));

fig = go.Figure(data=traces, frames=frames, layout=layout)
fig.show()

In [384]:
fig.write_json("../data/json/new_vs_total_anim.json")